In [ ]:
import pandas as pd
import numpy as np
import gensim
from random import sample

In [ ]:
# Load data
train_chal= pd.read_csv('/content/drive/My Drive/Colab Notebooks/RNN/train.csv')
test_chal = pd.read_csv('/content/drive/My Drive/Colab Notebooks/RNN/test.csv')
chall_details= pd.read_csv('/content/drive/My Drive/Colab Notebooks/RNN/challenge_data.csv')

In [ ]:
train=train_chal.groupby('user_id').challenge.apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
test=test_chal.groupby('user_id').challenge.apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
set(train.user_id).intersection(test.user_id
                               )

set()

In [ ]:
train.shape, test.shape

((69532, 2), (39732, 2))

In [ ]:
sentence_list=pd.concat((train.challenge,test.challenge)).apply(lambda x: x.split())
# sentence_list= train.challenge.apply(lambda x: x.split())

In [ ]:
# train word2vec model to learn embeddings
neg=15
itera=200
window=5
size=500
print ('doing_'+str(neg)+'_'+str(window)+'_'+str(itera))
model=gensim.models.Word2Vec(sentence_list,size=size,window=window,workers=8,min_count=0, hs=1,iter=itera,sg=1)
model.save('word2vec_solved_iter')

"neg=15\nitera=200\nwindow=5\nsize=500\nprint ('doing_'+str(neg)+'_'+str(window)+'_'+str(itera))\nmodel=gensim.models.Word2Vec(sentence_list,size=size,window=window,workers=8,min_count=0, hs=1,iter=itera,sg=1)\nmodel.save('word2vec_solved_iter')"

In [ ]:
model= gensim.models.Word2Vec.load('/content/drive/My Drive/Colab Notebooks/RNN/word2vec_solved_iter__noshuf_5window_100_size_500')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
model.most_similar('CI23855')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('CI23663', 0.36745941638946533),
 ('CI23975', 0.31903666257858276),
 ('CI23933', 0.31583738327026367),
 ('CI23913', 0.2977244257926941),
 ('CI25135', 0.2900288701057434),
 ('CI23714', 0.2872004508972168),
 ('CI24915', 0.2844502329826355),
 ('CI24917', 0.2660750150680542),
 ('CI24958', 0.2638528645038605),
 ('CI24527', 0.2628430724143982)]

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import TimeDistributed, concatenate, Bidirectional, Masking, RepeatVector
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers.convolutional import Convolution1D, MaxPooling1D


In [ ]:
pretrained_weights = model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

Result embedding shape: (5502, 500)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [ ]:
def word2idx(word):
  return model.wv.vocab[word].index
def idx2word(idx):
  return model.wv.index2word[idx]

In [ ]:
# generate int sequences for challenge sentences
train_seq=np.array(train.challenge.apply(lambda x: x.split()).apply(lambda x: ([(word2idx(y)) for y in x])).tolist())
test_seq=np.array(test.challenge.apply(lambda x: x.split()).apply(lambda x: ([(word2idx(y)) for y in x])).tolist())

In [ ]:
print('train:',train_seq.shape)
print('------------------')
print('test:',test_seq.shape)
print('------------------')
print('train:',train_seq)
print('------------------')
print('test:',test_seq)

train: (69532, 13)
------------------
test: (39732, 10)
------------------
train: [[   1    0    2 ...    4   36    8]
 [   3    0    5 ...   13  101   33]
 [ 209  260  228 ...  340  233  188]
 ...
 [  60  143   33 ...   68  524  802]
 [  48    5   58 ...  351  464 1170]
 [  13  143  246 ... 2499 1270 2247]]
------------------
test: [[  0   5   2 ...   6 101   7]
 [  3   0   2 ...  13   4  93]
 [369 702 549 ... 595 479 266]
 ...
 [  5 123  29 ... 699 468  68]
 [  3 123 143 ... 299 387   6]
 [ 12   5  83 ...   1   7  62]]


In [ ]:
xtrain=train_seq[:,:10]
ytrain= train_seq[:,10]
xtrain.shape, ytrain.shape

((69532, 10), (69532,))

In [ ]:
print('xtrain:',xtrain.shape)
print('------------------')
print('ytrain:',ytrain.shape)
print('------------------')
print('train:',xtrain)
print('------------------')
print('ytrain:',ytrain)

xtrain: (69532, 10)
------------------
ytrain: (69532,)
------------------
train: [[   1    0    2 ...   10   13   35]
 [   3    0    5 ...   93   36    6]
 [ 209  260  228 ...  210  179  202]
 ...
 [  60  143   33 ...  212  305    8]
 [  48    5   58 ...   11  151  620]
 [  13  143  246 ... 2817  205  166]]
------------------
ytrain: [   4   13  340 ...   68  351 2499]


In [ ]:
# add 11th,12th,13th challenge to outcome
xtrain= np.concatenate((xtrain,xtrain,xtrain),axis=0)
ytrain= np.concatenate((train_seq[:,10],train_seq[:,11],train_seq[:,12]))

In [ ]:
print('xtrain:',xtrain.shape)
print('------------------')
print('ytrain:',ytrain.shape)
print('------------------')
print('train:',xtrain)
print('------------------')
print('ytrain:',ytrain)

xtrain: (208596, 10)
------------------
ytrain: (208596,)
------------------
train: [[   1    0    2 ...   10   13   35]
 [   3    0    5 ...   93   36    6]
 [ 209  260  228 ...  210  179  202]
 ...
 [  60  143   33 ...  212  305    8]
 [  48    5   58 ...   11  151  620]
 [  13  143  246 ... 2817  205  166]]
------------------
ytrain: [   4   13  340 ...  802 1170 2247]


In [ ]:
# define nn model
from keras.layers import Embedding
emdedding_size=500
vocab_size=5502
keras_model2 = Sequential()
keras_model2.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, 
                    weights=[pretrained_weights]))

keras_model2.add(GRU(units=emdedding_size))
keras_model2.add(Dense(units=vocab_size,activation="relu"))

keras_model2.add(Activation('softmax'))
keras_model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
sample_weight=np.concatenate((np.ones(train.shape[0])*3,np.ones(train.shape[0])*2,np.ones(train.shape[0])))

In [ ]:
'''# fit differnt samples of data and then full data
keras_model2.fit(xtrain,ytrain, epochs=5, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.10)
keras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.10)
keras_model2.fit(xtrain,ytrain, epochs=5, sample_weight=sample_weight,
          batch_size=256, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,
          batch_size=256, verbose=1,validation_split=0.10)
keras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,
          batch_size=256, verbose=1,validation_split=0.10)
keras_model2.fit(xtrain,ytrain, epochs=5, sample_weight=sample_weight,
          batch_size=1024, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=10, sample_weight=sample_weight,
          batch_size=512, verbose=1)'''

'# fit differnt samples of data and then full data\nkeras_model2.fit(xtrain,ytrain, epochs=5, sample_weight=sample_weight,\n          batch_size=512, verbose=1,validation_split=0.11)\nkeras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,\n          batch_size=512, verbose=1,validation_split=0.10)\nkeras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,\n          batch_size=512, verbose=1,validation_split=0.10)\nkeras_model2.fit(xtrain,ytrain, epochs=5, sample_weight=sample_weight,\n          batch_size=256, verbose=1,validation_split=0.11)\nkeras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,\n          batch_size=256, verbose=1,validation_split=0.10)\nkeras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,\n          batch_size=256, verbose=1,validation_split=0.10)\nkeras_model2.fit(xtrain,ytrain, epochs=5, sample_weight=sample_weight,\n          batch_size=1024, verbose=1,validation_split=0.11)\nkeras_model2.fit(xtrain

In [ ]:
'''# fit differnt samples of data and then full data
keras_model2.fit(xtrain,ytrain, epochs=5, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=3, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=3, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=5, sample_weight=sample_weight,
          batch_size=256, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=3, sample_weight=sample_weight,
          batch_size=256, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=3, sample_weight=sample_weight,
          batch_size=256, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=5, sample_weight=sample_weight,
          batch_size=1024, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=3, sample_weight=sample_weight,
          batch_size=1024, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=3, sample_weight=sample_weight,
          batch_size=1024, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=10, sample_weight=sample_weight,
          batch_size=512, verbose=1)'''

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 185650 samples, validate on 22946 samples
Epoch 1/5
185650/185650 [==============================] - 527s 3ms/step - loss: 11.6608 - accuracy: 0.0372 - val_loss: 5.4183 - val_accuracy: 0.0305
Epoch 2/5
185650/185650 [==============================] - 524s 3ms/step - loss: 9.7055 - accuracy: 0.0644 - val_loss: 5.1837 - val_accuracy: 0.0370
Epoch 3/5
185650/185650 [==============================] - 528s 3ms/step - loss: 8.8673 - accuracy: 0.0907 - val_loss: 5.1011 - val_accuracy: 0.0385
Epoch 4/5
185650/185650 [==============================] - 530s 3ms/step - loss: 8.1323 - accuracy: 0.1179 - val_loss: 5.1336 - val_accuracy: 0.0372
Epoch 5/5
185650/185650 [==============================] - 536s 3ms/step - loss: 7.4518 - accuracy: 0.1466 - val_loss: 5.2305 - val_accuracy: 0.0345
Train on 185650 samples, validate on 22946 samples
Epoch 1/3
185650/185650 [==============================] - 527s 3ms/step - loss: 6.8631 - accuracy: 0.1708 - val_loss: 5.3602 - val_accuracy: 0.0302
Epo

In [ ]:
# fit differnt samples of data and then full data
keras_model2.fit(xtrain,ytrain, epochs=2,
          batch_size=256, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=2,
          batch_size=512, verbose=1,validation_split=0.10)
keras_model2.fit(xtrain,ytrain, epochs=2, 
          batch_size=768, verbose=1,validation_split=0.10)
keras_model2.fit(xtrain,ytrain, epochs=2, 
          batch_size=1024, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=2, 
          batch_size=512, verbose=1)

Train on 185650 samples, validate on 22946 samples
Epoch 1/2
185650/185650 [==============================] - 645s 3ms/step - loss: 7.0925 - accuracy: 0.0453 - val_loss: 7.2894 - val_accuracy: 0.0302
Epoch 2/2
185650/185650 [==============================] - 654s 4ms/step - loss: 6.5929 - accuracy: 0.0711 - val_loss: 7.0854 - val_accuracy: 0.0402
Train on 187736 samples, validate on 20860 samples
Epoch 1/2
187736/187736 [==============================] - 557s 3ms/step - loss: 6.2731 - accuracy: 0.1012 - val_loss: 6.9965 - val_accuracy: 0.0400
Epoch 2/2
187736/187736 [==============================] - 562s 3ms/step - loss: 6.0843 - accuracy: 0.1190 - val_loss: 6.9667 - val_accuracy: 0.0394
Train on 187736 samples, validate on 20860 samples
Epoch 1/2
187736/187736 [==============================] - 524s 3ms/step - loss: 5.8741 - accuracy: 0.1430 - val_loss: 6.9522 - val_accuracy: 0.0388
Epoch 2/2
187736/187736 [==============================] - 530s 3ms/step - loss: 5.7130 - accuracy: 0.

In [ ]:
#keras_model2.evaluate(xtrain, ytrain)

val loss= 5.0348, 5.07

In [ ]:
import pickle
pickle.dump(keras_model2.to_json(),open('gru5.pkl','wb'))

In [ ]:
import h5py
import keras
# from importlib import reload
# import keras
keras_model2.save_weights('gru5.weights')

In [ ]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return preds.argsort()[-3:][::-1]
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
#     print max(probas[0])
    retar= probas[0].argsort()[-3:][::-1]
    return retar

def generate_next(text):
    global count
    count = 0
    count+=1
    if count%100==0:
        print (count)
    word_idxs = [word2idx(word) for word in text.split()]
    prediction = keras_model2.predict(x=np.array(word_idxs))
#     print prediction
    idxl = sample(prediction[-1], temperature=0.0)
#     word_idxs.e(idx)
#     print idxl
    return [idx2word(idx) for idx in idxl]



In [ ]:
test.challenge.values[0]

'CI23855 CI23933 CI24917 CI24915 CI23714 CI23663 CI24958 CI25135 CI25727 CI24530'

In [ ]:
generate_next('CI23855 CI23933 CI24917 CI24915 CI23714 CI23663 CI24958 CI25135 CI25727 CI24530')

['CI25509', 'CI27032', 'CI23763']

In [ ]:
test_inds=np.array(test.challenge.apply(lambda x: np.array([word2idx(word) for word in x.split()])).tolist())

In [ ]:
test_inds.shape, xtrain.shape

((39732, 10), (208596, 10))

In [ ]:
# make predictions
nnpred=keras_model2.predict(test_inds)

In [ ]:
nnpred.shape

(39732, 5502)

In [ ]:
nnpred2= pd.Series(nnpred.argsort(axis=1)[:,-3:].tolist())

In [ ]:
# store to csv
count=0
test['pred']=nnpred2.apply(lambda x: [idx2word(y) for y in x[::-1]])
test_sub= test.copy()
test_sub=pd.concat((test_sub,test_sub,test_sub))
test_sub=test_sub.sort_values('user_id').reset_index(drop=True)
test_sub['seq']=test_sub.groupby('user_id').cumcount()
test_sub.seq= test_sub.seq+11
test_sub['user_sequence']= test_sub.user_id.astype('str') + '_' + test_sub.seq.astype('str')
test_sub['challenge']=test_sub.apply(lambda row: row['pred'][row['seq']-11], axis=1)
test_sub[['user_sequence','challenge']].to_csv('saurabh10.csv',index=False)

In [ ]:
# store to csv
count=0
test['pred']=nnpred2.apply(lambda x: [idx2word(y) for y in x[::-1]])
test_sub= test.copy()
test_sub=pd.concat((test_sub,test_sub,test_sub))
test_sub=test_sub.sort_values('user_id').reset_index(drop=True)
test_sub['seq']=test_sub.groupby('user_id').cumcount()
test_sub.seq= test_sub.seq+11
test_sub['user_sequence']= test_sub.user_id.astype('str') + '_' + test_sub.seq.astype('str')
test_sub['challenge']=test_sub.apply(lambda row: row['pred'][row['seq']-11], axis=1)
test_sub[['user_sequence','challenge']].to_csv('/content/drive/My Drive/Colab Notebooks/RNN/saurabh10.csv',index=False)

In [ ]:
import pickle
pickle.dump(keras_model2.to_json(),open('/content/drive/My Drive/Colab Notebooks/RNN/gru5.pkl','wb'))

In [ ]:
import h5py
import keras
# from importlib import reload
# import keras
keras_model2.save_weights('/content/drive/My Drive/Colab Notebooks/RNN/gru5.weights')